In [1]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:

nltk.download('vader_lexicon')


def fetch_data_from_sql():
    # Definimos la cadena de conexión con los parámetros para la conexión a la base de datos

    conn_str = (
        "Driver={SQL Server};"
        "Server=192.168.1.132;"
        "Database=PortfolioProject_MarketingAnalytics;" 
        "UID=sa;"  
        "PWD=VeryStr0ngP@ssw0rd;"  
    )
    # Establecemos la conexión a la base de datos
    conn = pyodbc.connect(conn_str)
    
   # Define la consulta SQL para obtener los datos de las reseñas de clientes
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    
   # Ejecuta la consulta y carga los datos en un DataFrame
    df = pd.read_sql(query, conn)
    
    # Cerramos la conexión para liberar recursos
    conn.close()
    
    # Devuelve los datos obtenidos como un DataFrame
    return df

# Obtenemos los datos de las reseñas de clientes desde la base de datos SQL
customer_reviews_df = fetch_data_from_sql()

# Iniciamos el analizador de intensidad de sentimiento VADER para analizar el sentimiento de los datos de texto
sia = SentimentIntensityAnalyzer()

# Definimos a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    # Obtenemos las puntuaciones de sentimiento para los textos de las reseñas
    sentiment = sia.polarity_scores(review)
    # Devolvemos la puntuación compuesta, que es una puntuación normalizada entre -1 (más negativa) y 1 (más positiva)

    return sentiment['compound']

# Definimos una función para categorizar el sentimiento utilizando tanto la puntuación de sentimiento como la calificación de la reseña
def categorize_sentiment(score, rating):
    # Utilizamos tanto la puntuación de sentimiento del texto como la calificación numérica para determinar la categoría de sentimiento
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment

# Definimos una función para agrupar las puntuaciones de sentimiento en rangos de texto
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment

# Aplicamos el análisis de sentimiento para calcular las puntuaciones de sentimiento de cada reseña
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

# Aplicamos la categorización de sentimiento utilizando tanto el texto como la calificación
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

# Aplicamos la agrupación de sentimiento para categorizar las puntuaciones en rangos definidos
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

# Mostramos las primeras filas del DataFrame con las puntuaciones de sentimiento, categorías y agrupaciones
print(customer_reviews_df.head())

# Guardamos el DataFrame con las puntuaciones de sentimiento, categorías y agrupaciones en un nuevo archivo CSV
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Patricia\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\Patricia\AppData\Local\Temp\ipykernel_5232\250592342.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2021-12-23       3   
1         2          80         19  2022-12-25       5   
2         3          50         13  2021-01-26       4   
3         4          78         15  2021-04-21       3   
4         5          64          2  2021-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  
